In [1]:
import os 
import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras.models import load_model
import pandas as pd
import helpers as H

Using TensorFlow backend.
C:\Users\Aiman\Anaconda3\envs\sticker\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [2]:
def dir_test(path):
    return len(os.listdir(path))

w, h = 128, 128
def rs_img(img):
    '''W and H is 128 now
    '''
    #H.normalize
    flatten = [cv2.resize(img[:,:,i], (w, h), interpolation=cv2.INTER_CUBIC) for i in range(img.shape[-1])]
    img = np.array(np.dstack(flatten)) 
    return img

def change_depth(img):
    '''Depth is 32 now
    '''
    #img_start = img[:,:,:8]
    
    mid = int(img.shape[-1]/2)
    img_middle = img[:,:,mid-16:mid+16]
    
    #img_end = img[:,:,-8:]
    #img = np.concatenate((img_start, img_middle, img_end), axis=2)
    img = img_middle
    
    return img

def get_number(filename):
    return int(filename[:filename.find('.')])

def sort_paths(paths):
    paths.sort(key = get_number)
    return paths

def load_numpy(path):
    val = np.load(path)
    print(val.shape)
    return val

def expand_dims(val):
    val_exp = np.expand_dims(val, axis=4)
    return val_exp

def plot_slice(img, sl_num):
    plt.imshow(img[:,:,sl_num])


def to_3D(arr, resize_dim=None):
    '''Stack 2D slice to add dummy 3-channel dimension'''
    img = np.asarray(np.dstack((arr, arr, arr)), dtype=np.float64)
    if resize_dim is not None:
        img=cv2.resize(img,(resize_dim, resize_dim),interpolation=cv2.INTER_AREA)    
    else:
        pass
    return img

In [3]:
test_data_path = "dataset/test_images/"

In [4]:
sort_paths(os.listdir(test_data_path))[:5]

['1.npy', '2.npy', '3.npy', '4.npy', '5.npy']

In [ ]:
x_test = []
c = 0

for path in sort_paths(os.listdir(test_data_path))[:]:
    img = None
    img = np.load(test_data_path+ "{}".format(path))
    
    # Segment lung and normalize
    img_rs = np.transpose(img, (2, 0, 1)) # CHW
    
    # Lung segmentation, padding if needed, threshold -1100 to 700, divide all vals with 1100 to bring to -1 to 1
    img_rs = H.seperate_lungs_and_pad(img_rs)
    img_rs = H.threshold_and_normalize_scan(img_rs)
    
    img_rs = np.transpose(img_rs, (1, 2, 0)) # HWC

    # Resize w, h 
    img_rs = rs_img(img_rs)
    
    # Resize depth
    img_rs = change_depth(img_rs)
    
    x_test.append(img_rs)
    c+=1
    print(c, "----", test_data_path+path, img_rs.shape)

x_test = np.array(x_test)
print(x_test.shape)
np.save("dataset/x_test.npy", x_test)
print("Saved!")

1 ---- dataset/test_images/1.npy (128, 128, 32)
2 ---- dataset/test_images/2.npy (128, 128, 32)
3 ---- dataset/test_images/3.npy (128, 128, 32)
4 ---- dataset/test_images/4.npy (128, 128, 32)
5 ---- dataset/test_images/5.npy (128, 128, 32)
6 ---- dataset/test_images/6.npy (128, 128, 32)
7 ---- dataset/test_images/7.npy (128, 128, 32)
8 ---- dataset/test_images/8.npy (128, 128, 32)
9 ---- dataset/test_images/9.npy (128, 128, 32)
10 ---- dataset/test_images/10.npy (128, 128, 32)
11 ---- dataset/test_images/11.npy (128, 128, 32)
12 ---- dataset/test_images/12.npy (128, 128, 32)
13 ---- dataset/test_images/13.npy (128, 128, 32)
14 ---- dataset/test_images/14.npy (128, 128, 32)
15 ---- dataset/test_images/15.npy (128, 128, 32)
16 ---- dataset/test_images/16.npy (128, 128, 32)
17 ---- dataset/test_images/17.npy (128, 128, 32)
18 ---- dataset/test_images/18.npy (128, 128, 32)
19 ---- dataset/test_images/19.npy (128, 128, 32)
20 ---- dataset/test_images/20.npy (128, 128, 32)
21 ---- dataset/te

In [ ]:
x_test = np.load("dataset/x_test.npy")
x_test.shape

In [11]:
x_test = expand_dims(x_test)
x_test.shape

(117, 128, 128, 32, 1)

In [12]:
model = None
model = load_model("resnet3D_exp1.h5")
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 32, 0                                            
__________________________________________________________________________________________________
conv3d_1 (Conv3D)               (None, 64, 64, 16, 6 22016       input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 64, 64, 16, 6 256         conv3d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 64, 64, 16, 6 0           batch_normalization_1[0

In [13]:
y_preds = []
y_preds_arg = []

count = len(x_test)

for i in range(count):
    y = model.predict(np.expand_dims(x_test[i], axis=0))
    y_preds.append(y)
    y_preds_arg.append(np.argmax(y))
    y = None
    
len(y_preds)

117

In [14]:
y_preds[:5]

[array([[0.2960737 , 0.70392627]], dtype=float32),
 array([[0.7859806 , 0.21401942]], dtype=float32),
 array([[0.35866696, 0.64133304]], dtype=float32),
 array([[0.27814388, 0.7218562 ]], dtype=float32),
 array([[0.280744, 0.719256]], dtype=float32)]

In [17]:
dt = pd.read_csv('TestSet_metaData.csv')
dt.head()

,Filename,md_Disability,md_Relapse,md_SymptomsOfTB,md_Comorbidity,md_Bacillary,md_DrugResistance,md_HigherEducation,md_ExPrisoner,md_Alcoholic,md_Smoking
0,CTR_TST_001.nii.gz,0,1,1,1,0,0,1,0,0,0
1,CTR_TST_002.nii.gz,0,1,0,0,1,4,0,0,0,1
2,CTR_TST_003.nii.gz,0,0,0,0,1,3,0,0,0,1
3,CTR_TST_004.nii.gz,0,1,1,0,0,0,0,0,1,1
4,CTR_TST_005.nii.gz,1,1,0,0,1,3,0,0,0,1


In [18]:
patient_names = dt['Filename'].values
len(patient_names)

117

In [19]:
patient_names[:3]

array(['CTR_TST_001.nii.gz', 'CTR_TST_002.nii.gz', 'CTR_TST_003.nii.gz'],
      dtype=object)

In [22]:
names = []

for name in patient_names:
    names.append(name[:-7])

names[:3]

['CTR_TST_001', 'CTR_TST_002', 'CTR_TST_003']

In [20]:
probab = []

for p in y_preds:
    # probability of HIGH severity as required to make submission
    probab.append(p[0][1])
    
probab[:5]

[0.70392627, 0.21401942, 0.64133304, 0.7218562, 0.719256]

In [23]:
len(names), len(probab)

(117, 117)

In [26]:
with open('results_texts/Sresnet_3d_exp1.txt', 'w') as f:
    for n, p in zip(names, probab):
        print(n,",", p)
        f.write(str(n))
        f.write(",")
        f.write(str(p))
        f.write("\n")
        
f.close()

CTR_TST_001 , 0.70392627
CTR_TST_002 , 0.21401942
CTR_TST_003 , 0.64133304
CTR_TST_004 , 0.7218562
CTR_TST_005 , 0.719256
CTR_TST_006 , 0.6906187
CTR_TST_007 , 0.6042552
CTR_TST_008 , 0.3325997
CTR_TST_009 , 0.15353563
CTR_TST_010 , 0.5895286
CTR_TST_011 , 0.7182313
CTR_TST_012 , 0.6739476
CTR_TST_013 , 0.4804707
CTR_TST_014 , 0.7396892
CTR_TST_015 , 0.30143026
CTR_TST_016 , 0.38166168
CTR_TST_017 , 0.49455228
CTR_TST_018 , 0.7544957
CTR_TST_019 , 0.6877154
CTR_TST_020 , 0.63146156
CTR_TST_021 , 0.6014201
CTR_TST_022 , 0.6170795
CTR_TST_023 , 0.24936743
CTR_TST_024 , 0.33587295
CTR_TST_025 , 0.429197
CTR_TST_026 , 0.6643837
CTR_TST_027 , 0.6625817
CTR_TST_028 , 0.4944353
CTR_TST_029 , 0.6925161
CTR_TST_030 , 0.1788668
CTR_TST_031 , 0.5354365
CTR_TST_032 , 0.57876164
CTR_TST_033 , 0.19841734
CTR_TST_034 , 0.76508075
CTR_TST_035 , 0.6793628
CTR_TST_036 , 0.6599217
CTR_TST_037 , 0.71639663
CTR_TST_038 , 0.67823625
CTR_TST_039 , 0.66354585
CTR_TST_040 , 0.4975978
CTR_TST_041 , 0.76416063
C

https://www.crowdai.org/challenges/imageclef-2019-tuberculosis-severity-scoring/submissions